# Imports

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from io import StringIO
import os 
import seaborn as sns
import mne  # Muy útil para EEG
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import antropy as ant
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score



# Carga de datos

In [19]:
# Ruta de las carpetas
channels_file = 'Data/channels.csv'
eeg_folder = 'Data/NBack/'

# Combinar todos los archivos EEG
all_eeg = []
eeg_files = [f for f in os.listdir(eeg_folder) if f.startswith('EEG_') and f.endswith('.csv')]

for eeg_file in eeg_files:
    with open(os.path.join(eeg_folder, eeg_file), 'r') as file:
        lines = file.readlines()
    lines[0] = lines[0].lstrip(',')  # Eliminar la coma inicial de la primera línea
    eeg_data = pd.read_csv(StringIO(''.join(lines)))
    eeg_data['source_file'] = eeg_file  # Agregar columna para identificar el archivo de origen
    all_eeg.append(eeg_data)

combined_eeg = pd.concat(all_eeg, ignore_index=True)

# Combinar todos los archivos NBack
all_nback = []
nback_files = [f for f in os.listdir(eeg_folder) if f.startswith('NBack_') and f.endswith('.csv')]

for nback_file in nback_files:
    nback_data = pd.read_csv(os.path.join(eeg_folder, nback_file))
    nback_data['source_file'] = nback_file  # Agregar columna para identificar el archivo de origen
    all_nback.append(nback_data)

combined_nback = pd.concat(all_nback, ignore_index=True)

# Leer los nombres de los canales desde el archivo channels.csv
channels = ['timestamp'] + pd.read_csv(channels_file, header=None).iloc[0].tolist()

# Cambiar los nombres de las columnas del DataFrame combinado de EEG
combined_eeg.columns = channels + ['source_file']  # Mantener la columna 'source_file' al final

# Mostrar un resumen del DataFrame combinado con los nuevos nombres de columnas
print("EEG combinado con nombres actualizados:")
print(combined_eeg.head())

# Opcional: Guardar los DataFrames combinados en archivos CSV
# combined_eeg.to_csv('combined_eeg.csv', index=False)
# combined_nback.to_csv('combined_nback.csv', index=False)

# Mostrar un resumen de los datos combinados
print("EEG combinado:")
print(combined_eeg.head())
print("\nNBack combinado:")
print(combined_nback.head())

EEG combinado con nombres actualizados:
   timestamp           F4            Oz            Fz            F8  \
0   0.045299  2327.497887  14123.593249  19397.060884  15709.190157   
1   0.049206  2370.036155  14146.383345  19430.449044  15774.822714   
2   0.053112  2344.581333  14114.430973  19402.548515  15743.916764   
3   0.057018  2391.159277  14158.879570  19451.718178  15794.388370   
4   0.060925  2308.747466  14097.809899  19366.471294  15694.175219   

            Pz            F5            T5           T3            C4  \
0  3360.109478  17359.494877  23611.987567  9074.315427  19640.499994   
1  3389.786490  17380.532825  23620.748308  9078.987823  19665.455941   
2  3350.107631  17357.085673  23599.345330  9150.229688  19653.215238   
3  3394.349377  17409.504111  23619.896570  9195.578694  19687.540310   
4  3320.880823  17326.848947  23567.526802  8976.560151  19600.456104   

             C3            T4            P4            T6           Cz  \
0  25970.245391  196

# Segmentación por ventanas deslizantes

In [9]:
fs = 256  # Hz
window_size = 2 * fs # 2 segundos
step_size = int(0.5 * fs) # 0.5 segundos de solapamiento

segments = []
labels = []

# Ejemplo: agrupar por bloques con mismo nivel de n-back (evento código 4)
nback_blocks = events[events[1] == 4]

for i in range(len(nback_blocks) - 1):
    n_val = int(events.iloc[nback_blocks.index[i], 2])
    start_time = nback_blocks.iloc[i, 0]
    end_time = nback_blocks.iloc[i+1, 0]

    eeg_block = eeg[(eeg['timestamp'] >= start_time) & (eeg['timestamp'] < end_time)]

    for start in range(0, len(eeg_block) - window_size, step_size):
        window = eeg_block.iloc[start:start + window_size, 1:]
        segments.append(window.values)
        labels.append(n_val)  # n (nivel de carga cognitiva)


KeyError: 1

# Extracción de características

In [ ]:
def extract_features(segment):
    features = []
    for channel in segment.T:
        features.append(np.mean(channel))
        features.append(np.std(channel))
        features.append(ant.petrosian_fd(channel))  # complejidad
        features.append(ant.spectral_entropy(channel, sf=fs))
    return features

X = np.array([extract_features(seg) for seg in segments])
y = np.array(labels)


# Preprocesamiento

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, test_size=0.2, random_state=42)

# Entrenamiento del modelo

## Random Forest

In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_pred_rf = clf.predict(X_test)

print(classification_report(y_test, y_pred_rf))


## SVM

In [ ]:
svm_model = SVC(kernel='rbf', C=1, gamma='scale')
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))


## KNN

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)

print("KNN Classification Report:")
print(classification_report(y_test, y_pred_knn))


# Visualización

## Matiz de confusión

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay

def plot_confusion(y_true, y_pred, model_name):
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap='Blues')
    plt.title(f'Matriz de confusión: {model_name}')
    plt.grid(False)
    plt.show()

plot_confusion(y_test, y_pred_rf, "Random Forest")
plot_confusion(y_test, y_pred_svm, "SVM")
plot_confusion(y_test, y_pred_knn, "KNN")


## Métricas

In [ ]:


def get_metrics(y_true, y_pred, model_name):
    return {
        "Modelo": model_name,
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, average='weighted'),
        "Recall": recall_score(y_true, y_pred, average='weighted'),
        "F1-score": f1_score(y_true, y_pred, average='weighted')
    }

results = [
    get_metrics(y_test, y_pred_rf, "Random Forest"),
    get_metrics(y_test, y_pred_svm, "SVM"),
    get_metrics(y_test, y_pred_knn, "KNN")
]

metrics_df = pd.DataFrame(results)

# Visualización en barra
metrics_df.set_index("Modelo")[["Accuracy", "Precision", "Recall", "F1-score"]].plot(kind='bar', figsize=(10,6), ylim=(0,1), colormap="Set2")
plt.title("Comparación de Métricas")
plt.ylabel("Valor")
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()


## Visualización 2D de los datos con PCA

In [ ]:

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Visualización con clases reales
plt.figure(figsize=(8,6))
sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=y, palette='Set2', alpha=0.7)
plt.title("Visualización PCA por clase real")
plt.xlabel("Componente 1")
plt.ylabel("Componente 2")
plt.legend(title="Carga Cognitiva (n)")
plt.grid(True)
plt.show()